# The Questions

In [16]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from sklearn.feature_extraction.text import CountVectorizer
 
from sklearn.feature_extraction.text import TfidfVectorizer
 
from sklearn.metrics.pairwise import cosine_similarity

path = os.path.realpath('../Data')

bool_df = pd.read_csv(path+"/recipeBool.csv")
bool_df.index = bool_df["Unnamed: 0"]
bool_df = bool_df.drop(["Unnamed: 0"], axis = 1)
columns = bool_df.columns

In [17]:
bool_df.head()


,57d13370b7e8697d4b30530e,57d13370b7e8697d4b30530f,57d13370b7e8697d4b305311,57d13370b7e8697d4b305313,57d13370b7e8697d4b305314,57d13370b7e8697d4b305315,57d13370b7e8697d4b305316,57d13370b7e8697d4b305317,57d13370b7e8697d4b305318,57d13370b7e8697d4b305319,...,57962a07b7e8697d4b3052f7,57962a07b7e8697d4b3052f8,57962a07b7e8697d4b3052f9,57962a07b7e8697d4b3052fa,57962a07b7e8697d4b3052fb,57962a07b7e8697d4b3052fc,57962a07b7e8697d4b3052fd,57962a07b7e8697d4b3052fe,57962a07b7e8697d4b3052ff,5a95b0bbae08b555554f4b52
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
53f45a29ff604dac328b456e,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
5469feafff604d2c8b8b4569,True,False,False,False,False,False,False,True,False,False,...,False,False,False,True,False,False,False,False,False,False
54c91e3e6ced6e5f178b4571,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
54c91e716ced6e39178b456c,False,False,False,False,False,False,True,False,False,True,...,False,True,False,False,False,False,False,True,False,False
54c91ed56ced6e5f178b4572,False,False,False,False,False,False,False,True,False,False,...,False,False,False,True,True,False,False,False,True,False


## Bei Einrichtung
1. Frage: Willst du einen Thermomix verwenden?
2. Frage: Nenne Zutaten, welche du nicht verwenden möchtest.
3. Frage: Hast du irgendwelche Alergien?
4. Frage: Besondere Ernährungsweise?
5. Frage: Wie bewertest du folgende 5 Gerichte? (0-5)


Funktion zu Erstellung eines Userprofiles:

In [18]:
#func to normalize text
def process_text(text):
    #replace muliple spaces with one
    test = ''.join(text.split())
    #lowercase
    text = text.lower()
    return text

In [20]:
#creating helping df´s for finden id´s

con = sqlite3.connect(path+"/RecipeDB.db")
# ing_df = pd.read_sql("Select * FROM IngFamily", con)
# temp = pd.read_sql('SELECT * FROM Ingredients',con)
# ing_df.index = ing_df["familyID"]
# for x in temp.itertuples():
#     ing_df.loc[x.familyID]['familyName']+= ' '+x.ingName
# ing_df = ing_df.drop(["familyID"], axis = 1)

# ing_df.rename(columns = {'familyName':'info'}, inplace = True)

ing_df = pd.read_sql("Select ingID, ingName, familyName FROM Ingredients NATURAL JOIN IngFamily", con)
ing_df.index = ing_df['ingID']
ing_df['info'] = ing_df['ingName'] +" "+ ing_df['familyName'];
ing_df = ing_df.drop(['ingName'], axis = 1)
ing_df = ing_df.drop(['familyName'], axis = 1)
ing_df = ing_df.drop(['ingID'], axis = 1)


ing_df['info'] = ing_df.apply(lambda x: process_text(x.info), axis = 1)


tags_df = pd.read_sql("Select * from Tags", con)
tags_df.index = tags_df["tagID"]
tags_df['info'] = tags_df['tagName']+' '+tags_df['tagType']
tags_df = tags_df.drop(["tagID"], axis = 1)
tags_df = tags_df.drop(["tagName"], axis = 1)
tags_df = tags_df.drop(["tagType"], axis = 1)

tags_df['info'] = tags_df.apply(lambda x: process_text(x.info), axis = 1)

alg_df = pd.read_sql("Select * from Allergens", con)
alg_df.index = alg_df["alleID"]
alg_df['info']= alg_df['alleType']+' '+alg_df['alleName']+' '+alg_df['alleSlug']
alg_df = alg_df.drop(["alleID"], axis = 1)
alg_df = alg_df.drop(["alleType"], axis = 1)
alg_df = alg_df.drop(["alleName"], axis = 1)
alg_df = alg_df.drop(["alleSlug"], axis = 1)

alg_df['info'] = alg_df.apply(lambda x: process_text(x.info), axis = 1)

thermo = tags_df[tags_df["info"].str.contains("thermomix")]




In [21]:
alg_df.head()

,info
alleID,
57962a07b7e8697d4b3052f5,gluten weizen weizen
57962a07b7e8697d4b3052f3,maycontaintracesofallergens kann spuren von al...
57962a07b7e8697d4b3052fa,milk milch laktose
57962a07b7e8697d4b3052fd,mustard senf senf
57962a07b7e8697d4b3052f6,egg ei ei


In [22]:
ing_df

,info
ingID,
554b7698fd2cb96c628b4568,arborio-reis reis
55a7a8f8fd2cb9570f8b4567,arabisches fladenbrot brot
556c4ed44dab710b3d8b4569,apfelessig essig
55535c2efd2cb9ae048b4567,aubergine aubergine
556db2a04dab71104b8b4568,naan-brot brot
...,...
5bf2c4e9ae08b5103d1caae2,schinkenbraten schweinefleisch
5c1227bee3f33930ed6f7d42,apfelsaft fruchtsaft
5c0e62fac445fa5e3d20aa72,hühnerbrühe stock


In [23]:
tags_df.head()

,info
tagID,
57d13370b7e8697d4b30531a,leichter genuss low-calorie
57d13370b7e8697d4b305319,scharf spicy
58aeb96c99052d5fee3dd712,chef's choice chefs-choice
57d13370b7e8697d4b305317,vegetarisch vegetarian
57d13370b7e8697d4b30530e,zeit sparen under-30-minutes


Gewichtungen für Userprofil Erstellung werden definiert

In [ ]:
test = (False, ["Schwein", "Tomaten", "Schimmelkäse"], ["Laktose", "Schwefel"], ["Protein", "fettarm"])
def create_Userprofile(tmix, ing, alg, tags):
    data = np.full([1, len(columns)], 0.25, dtype=np.float64)
    user_Profile = pd.DataFrame(data=data, columns = columns)
    for x in thermo.itertuples():
        user_Profile[x.Index] = 1.0 if tmix else 0.0
    for x in ing:
        for i in ing_df[ing_df['info'].str.contains(x.lower())].itertuples():
            print('ing :',x,' id info', i.Index, ' ', i.info)
            user_Profile[i.Index] = 0;
            
    for a in alg:
        for i in alg_df[alg_df['info'].str.contains(a.lower())].itertuples():
            print('allergie :',a,' id info', i.Index, ' ', i.info)
            user_Profile[i.Index] = -1.0
            
    for tag in tags:
        for i in tags_df[tags_df['info'].str.contains(tag.lower())].itertuples():
            print('tag :',tag,' id info', i.Index, ' ', i.info)
            user_Profile[i.Index] = 5.0
    return user_Profile.iloc[0]

user_p = create_Userprofile(test[0], test[1], test[2], test[3])

## Bei Normaler Verwendung
### 1. Option:
Empfehlung eines Rezeptes

### 2. Option:
Spezifischere Empfehlung durch folgende Frage:
1. Nenne Zutaten, welche du gerne verwenden möchtest.

In [25]:
bool_df.shape

(1808, 871)

In [26]:
user_p.shape

(871,)

## Knowledge Base Recommender

In [27]:
def know_recommender(user_profile):
    res = bool_df.dot(user_profile)
    for i in res.nlargest(1808).iteritems():
        yield i
        

In [28]:
#print(user_p)
count  = 0;
for i in know_recommender(user_p):
    count += 1
    print(i)
    if count == 5:
        break

('58f9e3cb043c3c42b53b8792', 14.5)
('5902ec12171c584090693d02', 13.75)
('58ac4befd0d6bd680533bac4', 13.25)
('5858e0e67d69bf305c3a7752', 12.0)
('5902ee9331310006be1c8b72', 10.5)


## Contentend based Recommender


In [29]:
recipe_df = pd.read_sql("SELECT recID FROM Recipe;", con)
cur = con.cursor()

for id in recipe_df.itertuples():
    recipe_df.at[id.Index, 'ingredients'] = ''.join([i[0].lower()+" " for i in cur.execute("Select ingName from Ingredients NATURAL JOIN IngInRec WHERE recID = ?", (id.recID,)) ])

for id in recipe_df.itertuples():
    recipe_df.at[id.Index, 'tags'] = ''.join([i[0].lower()+" " for i in cur.execute("Select tagName from Tags NATURAL JOIN TagsInRec WHERE recID = ?", (id.recID,)) ])

for id in recipe_df.itertuples():
    recipe_df.at[id.Index, 'allergies'] = ''.join([i[0].lower()+" " for i in cur.execute("Select alleName from Allergens NATURAL JOIN AlleInRec WHERE recID = ?", (id.recID,)) ])






In [31]:

def concatenate_features(ing, tags, allergies):
    w_ing = 3
    w_tag = 1
    w_allergies = 3;
    return process_text((ing*w_ing+tags*w_tag+allergies*w_allergies))

recipe_df['info'] = recipe_df.apply(lambda x : concatenate_features(x.ingredients, x.tags, x.allergies), axis = 1)
    

In [32]:
recipe_df.iloc[1]['info']

'öl* salz* zucker* mozzarella basilikum frische gnocchi schalotte stückige tomaten italienische kräutermischung pfeffer* knoblauchzehe hartkäse ital. art bunte kirschtomaten frischer majoran öl* salz* zucker* mozzarella basilikum frische gnocchi schalotte stückige tomaten italienische kräutermischung pfeffer* knoblauchzehe hartkäse ital. art bunte kirschtomaten frischer majoran öl* salz* zucker* mozzarella basilikum frische gnocchi schalotte stückige tomaten italienische kräutermischung pfeffer* knoblauchzehe hartkäse ital. art bunte kirschtomaten frischer majoran vegetarisch zeit sparen ei weizen milch kann spuren von allergenen enthalten ei weizen milch kann spuren von allergenen enthalten ei weizen milch kann spuren von allergenen enthalten '

In [33]:
#saving the recipe csv
output_file = os.path.join(path,'recipe_feature_set.csv')
recipe_csv = recipe_df.to_csv(output_file, index = True)


In [34]:

vect = CountVectorizer()
 
vect_matrix = vect.fit_transform(recipe_df['info'])
 
cosine_similarity_matrix_count_based = cosine_similarity(vect_matrix, vect_matrix)

In [35]:
print(cosine_similarity_matrix_count_based)

[[1.         0.36927447 0.39073029 ... 0.37367949 0.40605178 0.41174614]
 [0.36927447 1.         0.3607168  ... 0.34976707 0.3460259  0.25341223]
 [0.39073029 0.3607168  1.         ... 0.36614634 0.47461415 0.34758389]
 ...
 [0.37367949 0.34976707 0.36614634 ... 1.         0.29179466 0.21369596]
 [0.40605178 0.3460259  0.47461415 ... 0.29179466 1.         0.25007089]
 [0.41174614 0.25341223 0.34758389 ... 0.21369596 0.25007089 1.        ]]


In [36]:
def index_to_recipeID(index, df):
    return df.loc[index]['recID']

print(index_to_recipeID(0,recipe_df))

53f45a29ff604dac328b456e


In [37]:

def CoBaRe(recipe_id, cosine_similarity_matrix, df):
    index = df[df['recID']== recipe_id].index.values[0]
    similarity_scores = list(enumerate(cosine_similarity_matrix[index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    #delete the first cuz it´s itself lol
    print(similarity_scores.pop(0))
    for i in similarity_scores:
        yield index_to_recipeID(i[0],df);

In [38]:

test_df = pd.read_csv(path+'/recipe_feature_set.csv')
test_df.head()

,Unnamed: 0,recID,ingredients,tags,allergies,info
0,0,53f45a29ff604dac328b456e,salz* butter* olivenöl* zucker* gelbe paprika ...,NaN,weizen milch kann spuren von allergenen enthal...,salz* butter* olivenöl* zucker* gelbe paprika ...
1,1,5469feafff604d2c8b8b4569,öl* salz* zucker* mozzarella basilikum frische...,vegetarisch zeit sparen,ei weizen milch kann spuren von allergenen ent...,öl* salz* zucker* mozzarella basilikum frische...
2,2,54c91e3e6ced6e5f178b4571,rinderhüftsteak rosmarin öl* salz* pfefferkörn...,leichter genuss,milch weizen kann spuren von allergenen enthal...,rinderhüftsteak rosmarin öl* salz* pfefferkörn...
3,3,54c91e716ced6e39178b456c,roter chili öl* salz* pfefferkörner zucker* ka...,scharf low carb,sesam erdnüsse weizen,roter chili öl* salz* pfefferkörner zucker* ka...
4,4,54c91ed56ced6e5f178b4572,salz* pfefferkörner olivenöl* essig* karotte r...,vegetarisch,weizen schalenfrüchte schwefel milch kann spur...,salz* pfefferkörner olivenöl* essig* karotte r...


## TODO Recipe to JSON parser

In [ ]:
import json 
con = sqlite3.connect(path+"/RecipeDB.db")
cur = con.cursor();

def recipe_json(recipe_id):
    res = {}
    res['id'] = recipe_id
    sql = """ SELECT * FROM Recipe WHERE recID = ? ;"""
    recs = cur.execute(sql,(recipe_id,)).fetchall()[0]
    #basic infos
    res['name'] = recs[1]
    res['headline'] = recs[2]
    res['disc'] = recs[3]
    #ingredients
    recs = cur.execute("SELECT ingID, ingName, familyID, familyName, amount, unit FROM IngInRec NATURAL JOIN Ingredients NATURAL JOIN IngFamily WHERE recID = ? ;",(recipe_id,)).fetchall()
    ingr = {}
    for ing in recs:
        ingr[ing[0]] = {'name':ing[1],'amount': ing[4], 'unit': ing[5], 'family' : {'id':ing[2], 'name': ing[3]}}
    res['ingredients'] = ingr
    
    #nutr
    recs = cur.execute("SELECT nType, amount, unit, nName FROM NutrInRec NATURAL JOIN Nutration WHERE recID = ? ;",(recipe_id,)).fetchall()
    nutr = {}
    for i in recs:
        nutr[i[0]] = {'amount' : i[1], 'unit' : i[2], 'name' : i[3]}
    res['nutration'] = nutr
    
    #allergene
    recs = cur.execute("SELECT alleID, alleName FROM AlleInRec NATURAL JOIN Allergens WHERE recID = ? ;",(recipe_id,)).fetchall()
    alle = {}
    for i in recs:
        alle[i[0]] = i[1]
    res['allergies'] = alle
    
    #tags
    
    recs = cur.execute("SELECT tagID, tagName FROM TagsInRec NATURAL JOIN Tags WHERE recID = ? ;",(recipe_id,)).fetchall()
    tags = {}
    for t in recs:
        tags[t[0]] = t[1]
    res['tags'] = tags
    
    #steps
    recs = cur.execute("SELECT stepIndex, stepDisc from StepsInRec WHERE recID = ?  ORDER BY stepIndex ;", (recipe_id,)).fetchall();
    
    res['steps'] = [i[1]for i in recs]
    
    return json.dumps(res, indent=2, ensure_ascii=False)
    

for i in know_recommender(user_p):
    print(recipe_json(i[0]))
    for j in CoBaRe(i[0],cosine_similarity_matrix_count_based, recipe_df):
        print("CBS___________________________________________________________________________________________________________________________")
        print(recipe_json(j))
        break
    break